## Step 1: Install Required Packages

In [ ]:
!pip install ultralytics

## Step 2: Import Libraries

In [ ]:
from ultralytics import YOLO
import os
from google.colab import drive
import shutil
from IPython.display import Image, display

## Step 3: Mount Google Drive

In [ ]:
drive.mount('/content/drive')
!mkdir -p /content/cotton_detection

## Step 4: Upload Your Dataset
**Option A: Upload from Google Drive**

In [ ]:
drive_dataset_path = '/content/drive/MyDrive/Combined_Cotton_Dataset'

if os.path.exists(drive_dataset_path):
    !cp -r "{drive_dataset_path}" /content/cotton_detection/
    print("Dataset copied from Google Drive")
else:
    print("Dataset not found. Please upload it to Google Drive first.")

**Option B: Upload as ZIP file**

In [ ]:
from google.colab import files
import zipfile

uploaded = files.upload()

for filename in uploaded.keys():
    if filename.endswith('.zip'):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('/content/cotton_detection/')
        print(f"Extracted {filename}")

## Step 5: Verify Dataset Structure

In [ ]:
dataset_path = '/content/cotton_detection/Combined_Cotton_Dataset'

print("Dataset Structure:")
!ls -la {dataset_path}

print("\nTrain Images:")
!ls {dataset_path}/train/images | wc -l

print("Valid Images:")
!ls {dataset_path}/valid/images | wc -l

print("Test Images:")
!ls {dataset_path}/test/images | wc -l

## Step 6: Update data.yaml Configuration

In [ ]:
yaml_content = f"""path: {dataset_path}
train: train/images
val: valid/images
test: test/images

names:
  0: cotton_ready
  1: cotton_not_ready

nc: 2
"""

yaml_path = f'{dataset_path}/data.yaml'
with open(yaml_path, 'w') as f:
    f.write(yaml_content)

print("data.yaml created successfully")
!cat {yaml_path}

## Step 7: Load Pre-trained YOLOv11 Model

In [ ]:
model = YOLO('yolo11n.pt')
print("Model loaded successfully")

## Step 8: Train the Model

In [ ]:
results = model.train(
    data=yaml_path,
    epochs=100,
    imgsz=640,
    batch=16,
    name='cotton_detection',
    patience=20,
    save=True,
    device=0,
    workers=2,
    project='/content/runs/detect',
    optimizer='Adam',
    lr0=0.01,
    augment=True,
    plots=True
)

print("Training completed!")

## Step 9: View Training Results

In [ ]:
print("Training Results:")
display(Image(filename='/content/runs/detect/cotton_detection/confusion_matrix.png'))
display(Image(filename='/content/runs/detect/cotton_detection/results.png'))
display(Image(filename='/content/runs/detect/cotton_detection/val_batch0_pred.jpg'))

## Step 10: Validate the Model

In [ ]:
best_model = YOLO('/content/runs/detect/cotton_detection/weights/best.pt')
metrics = best_model.val()

print(f"\nValidation Metrics:")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")

## Step 11: Test Predictions on Sample Images

In [ ]:
test_results = best_model.predict(
    source=f'{dataset_path}/test/images',
    conf=0.25,
    save=True,
    project='/content/runs/detect',
    name='test_predictions'
)

print("Test predictions saved to: /content/runs/detect/test_predictions")

## Step 12: Export Model for Deployment

In [ ]:
best_model.export(format='onnx')
print("Model exported successfully")

## Step 13: Save Model to Google Drive

In [ ]:
drive_save_path = '/content/drive/MyDrive/Cotton_Detection_Models'
!mkdir -p "{drive_save_path}"

!cp /content/runs/detect/cotton_detection/weights/best.pt "{drive_save_path}/cotton_best.pt"
!cp /content/runs/detect/cotton_detection/weights/last.pt "{drive_save_path}/cotton_last.pt"
!cp -r /content/runs/detect/cotton_detection "{drive_save_path}/training_results"

print(f"Models saved to Google Drive: {drive_save_path}")

## Step 14: Download Model to Your Computer

In [ ]:
from google.colab import files

files.download('/content/runs/detect/cotton_detection/weights/best.pt')
print("Download complete!")

## Step 15: Test with Custom Image

In [ ]:
from google.colab import files
uploaded = files.upload()

for filename in uploaded.keys():
    results = best_model.predict(source=filename, conf=0.25, save=True)
    display(Image(filename=f'/content/runs/detect/predict/{filename}'))
    
    for r in results:
        print(f"\nDetections in {filename}:")
        for box in r.boxes:
            cls = int(box.cls[0])
            conf = float(box.conf[0])
            class_name = r.names[cls]
            print(f"  {class_name}: {conf:.2%}")

## Training Complete!

Your cotton detection model is now trained and ready to use.

**Model Files:**
- `best.pt` - Best performing model (use this for deployment)
- `last.pt` - Last epoch checkpoint